train 데이터에서 1년 기준의 n주차의 i요일인 날짜의 데이터를 sample_submission 데이터의 n주차 i요일에 넣고 평균

점수: 3.8801903775

In [1]:
!pip install pyarrow
!pip install fastparquet

In [106]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from datetime import datetime
import gc
import os
print(os.getcwd())
# os.chdir("./coding./jupyter./ddaLeung")

C:\Users\ab516\coding\jupyter\ddaLeung


In [107]:
def csv_to_parquet(csv_path, save_name):
    df = pd.read_csv(csv_path)
    df.to_parquet(f'./{save_name}.parquet')
    del df
    gc.collect()
    print(save_name, 'Done.')
    
csv_to_parquet('./open/train.csv', 'train')
csv_to_parquet('./open/sample_submission.csv', 'sample_submission')

train = pd.read_parquet('./train.parquet')
sample_submission=pd.read_parquet('./sample_submission.parquet')

train Done.
sample_submission Done.


In [108]:
df=pd.DataFrame({"일시":train['일시'].unique()})
df.sort_values(by="일시", ascending=True)

sample_submission.shape

(334, 5)

In [109]:
def make_month(df): # O
    dt = df['일시'].astype('str')
    month_data = pd.to_datetime(dt)
    md = month_data.dt.month
    return md

def make_weekOfYear(df):
    dt=df['일시'].astype('str')
    week_data=pd.to_datetime(dt)
    woy=week_data.dt.weekofyear
    return woy
    
def make_dayOfWeek(df):
    dt=df['일시'].astype('str')
    day_data=pd.to_datetime(dt)
    dow=day_data.dt.weekday
    return dow

In [110]:
train['month']=make_month(train)
train['weekOfYear']=make_weekOfYear(train)
train['dayOfWeek']=make_dayOfWeek(train)

sample_submission2=sample_submission.copy()
sample_submission2['month']=make_month(sample_submission2)
sample_submission2['weekOfYear']=make_weekOfYear(sample_submission2)
sample_submission2['dayOfWeek']=make_dayOfWeek(sample_submission2)
sample_submission2['개수']=0

C:\Users\ab516\AppData\Local\Temp\ipykernel_7504\2806905403.py:10: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  woy=week_data.dt.weekofyear
C:\Users\ab516\AppData\Local\Temp\ipykernel_7504\2806905403.py:10: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  woy=week_data.dt.weekofyear


In [113]:
train

,일시,광진구,동대문구,성동구,중랑구,month,weekOfYear,dayOfWeek
0,20180101,0.592,0.368,0.580,0.162,1,1,0
1,20180102,0.840,0.614,1.034,0.260,1,1,1
2,20180103,0.828,0.576,0.952,0.288,1,1,2
3,20180104,0.792,0.542,0.914,0.292,1,1,3
4,20180105,0.818,0.602,0.994,0.308,1,1,4
...,...,...,...,...,...,...,...,...
1456,20211227,3.830,3.416,2.908,2.350,12,52,0
1457,20211228,4.510,3.890,3.714,2.700,12,52,1
1458,20211229,4.490,3.524,3.660,2.524,12,52,2
1459,20211230,4.444,3.574,3.530,2.506,12,52,3


In [114]:
sample_submission2

,일시,광진구,동대문구,성동구,중랑구,month,weekOfYear,dayOfWeek,개수
0,20220101,0,0,0,0,1,52,5,0
1,20220102,0,0,0,0,1,52,6,0
2,20220103,0,0,0,0,1,1,0,0
3,20220104,0,0,0,0,1,1,1,0
4,20220105,0,0,0,0,1,1,2,0
...,...,...,...,...,...,...,...,...,...
329,20221126,0,0,0,0,11,47,5,0
330,20221127,0,0,0,0,11,47,6,0
331,20221128,0,0,0,0,11,48,0,0
332,20221129,0,0,0,0,11,48,1,0


In [115]:
print(sample_submission2.iloc[1,6])

52


In [116]:
for i in range(len(train)):
    for n in range(len(sample_submission2)): 
        if((train.iloc[i,6]==sample_submission2.iloc[n,6])and(train.iloc[i,7]==sample_submission2.iloc[n,7])):
                sample_submission2.iloc[n,8]+=1
                for t in range(4):
                    sample_submission2.iloc[n,t+1]+=train.iloc[i,t+1]

today = datetime.today().strftime('%y%H%M%S')
sample_submission2.to_csv("./submit_"+today+".csv", index = False)

train 데이터의 n주차 i요일을 sample_submission2 데이터 중 n주차 i요일 데이터에 더하고
이후 평균을 구하기 위해 sample_submission2 해당 일자의 '개수' column에 +1

In [117]:
for n in range(len(sample_submission2)):
    num=sample_submission2.iloc[n,8]
    for t in range(4):
        sample_submission2.iloc[n,t+1]=sample_submission2.iloc[n,t+1]/num

In [119]:
today = datetime.today().strftime('%y%H%M%S')
sample_submission2.drop(['month','weekOfYear','dayOfWeek','개수'],axis=1,inplace=True)
sample_submission2.to_csv("./submit_"+today+".csv", index = False)